In [27]:
import urllib2
import ssl

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

links = [ "https://199.119.112.144/onionperf-one-51200-2017-01-19.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-20.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-21.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-22.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-23.tpf",
          "https://199.119.112.144/onionperf-one-51200-2017-01-24.tpf",         
        ]

days = []

for link in links:
    f = urllib2.urlopen(link, context=ctx)
    myfile = f.read()
    days.append(myfile)

In [36]:
day_ps = []
for text in days:
    data = text.split('@type torperf 1.0')
    data_points = []
    for d in data:
        dp = d.split(' ')
        data_points.append(dp)
    data_points.pop(0)

    dps = []
    for dp in data_points:
        p = {}
        connect = 0
        complete = 0
        timeout = 0
        for d in dp:
            f = d.split('=')
            if f[0] == 'CONNECT':
                connect = f[1]
                p['connect'] = f[1]
            if f[0] == 'DATACOMPLETE':
                complete = f[1]
                p['complete'] = f[1]
            if f[0] == 'DIDTIMEOUT':
                timeout = f[1]
        if timeout == '0':
            transfer = float(complete) - float(connect)    
            if transfer < 30 and transfer > 0:
                p['transfer'] = transfer   
                dps.append(p)
    day_ps.append(dps)


In [56]:
transfer = []

for day in day_ps:
    t = []
    for dp in dps:
        t.append(dp['transfer'])
    transfer.append(t)

t = np.median(transfer, axis=0)

In [57]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

M = len(links)
# prepare some data
N = len(transfer)
x = np.linspace(0, M, N)
# output to static HTML file
output_file("transfer_times.html")

# create a new plot
s1 = figure(width=2048, plot_height=1024, tools=TOOLS)
s1.line(x, t, line_dash=(1, 2), line_color="green", line_width=2)
# NEW: put the subplots in a gridplot
p = gridplot([[s1]], toolbar_location=None)

# show the results
show(p)

INFO:bokeh.core.state:Session output file 'transfer_times.html' already exists, will be overwritten.
/usr/local/lib/python2.7/dist-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
